In [12]:
import os
import pickle

from dotenv import load_dotenv
from pydantic import BaseModel

from coop import AsyncHandler, AsyncSeqLikeHandler, async_
from eff import Handler, Operation, coroutine_decorator
from llm import (
	AsyncOneRoundChatHandler,
	AsyncReplayLLMHandler,
	LLMHandler,
	OneRoundChatHandler,
	ReplayLLMHandler,
	TraceLLMHandler,
	complete,
	parse,
)
from util import Timer, awaitable_args_decorator

In [13]:
# TODO 注意这个！！！
import nest_asyncio
nest_asyncio.apply()

In [ ]:
load_dotenv()

#BASE_URL = os.getenv("BASE_URL")
#API_KEY = os.getenv("API_KEY")

#TODO lazy way!! will change!!!
BASE_URL='https://api.openai-proxy.org/v1'
API_KEY=''

MODEL = "gpt-4o-mini"
TEMPERATURE = 1.0
MAX_TOKENS = 1000

In [15]:
import pandas as pd
from openai import OpenAI
import json
import time


In [16]:
client = OpenAI(
        base_url=BASE_URL,
        api_key=API_KEY,
    )

In [17]:
df = pd.read_excel('subsidy.xlsx')  
df.head()


,证券代码,证券简称,统计截止日期,列报会计科目,报表类型,项目,本期计入当期损益的金额,上期计入当期损益的金额,与资产相关/与收益相关
0,6,深振业A,2015-12-31,2,1,长沙市岳麓区天顶街道办事处税收奖励金,266250.0,210000.0,NaN
1,6,深振业A,2015-12-31,2,1,长沙市岳麓区财政局税收奖励金,1660000.0,NaN,NaN
2,6,深振业A,2015-12-31,2,1,西安浐灞生态区生态宜居示范单位奖励资金,150000.0,NaN,NaN
3,6,深振业A,2015-12-31,2,1,合计,2076250.0,210000.0,NaN
4,6,深振业A,2016-12-31,2,1,长沙市岳麓区天顶街道办事处税收奖励金,NaN,266250.0,NaN


In [18]:
df.shape

(700672, 9)

In [19]:
df = df[~df['项目'].astype(str).str.contains('合计', na=False)]
df.shape

(654054, 9)

In [20]:
df_short = df[0:10]

In [21]:
df_short.shape

(10, 9)

In [28]:
PROMPT_CLASSIFY = '''你是一名企业财务分析助理，任务是根据补贴或奖励项目名称，判断其属于以下补贴类型之一，并说明理由。

请严格按照以下分类标准执行：

1. 税收优惠和奖励：与退税、税收减免、税费返还、税收奖励等相关。例如：“税收奖励金”“减免税款”“税收返还”“即征即退”。
2. 研发补贴：与生产技术改造、新技术研发、创新资助、技术应用推广、产业化、专利相关。例如：“科技三项经费”“专利资助”“创新项目支持资金”。
3. 吸纳人才补贴：与引进人才、研究人员、科研团队、博士后、人才培养相关。例如：“博士后科研资助”“人才引进补贴”“科研人员经费”。
4. 产业升级补贴：与工业发展、产业扶持、生产设备更新、产业转型升级相关。例如：“产业发展奖励资金”“生产线改造补贴”“工业转型扶持资金”。
5. 项目运营补贴：与企业日常生产经营、土地整理、安全生产、电费、招商引资等有关。例如：“安全生产补贴”“招商引资奖励”“土地补偿资金”。
6. 就业补贴：与稳岗、就业、技能培训、职工补贴等有关。例如：“稳岗补贴”“就业奖励金”“技能培训补贴”。
7. 贸易补贴：与进出口业务、外贸发展、对外合作、市场拓展相关。例如：“外贸高质量发展资金”“出口奖励资金”“贸易促进基金”。
8. 环境保护和治理补贴：与节能环保、清洁生产、绿色发展、污染治理相关。例如：“节能专项资金”“环保设备改造补贴”“绿色生产补助”。
9. 信贷补贴：与贷款贴息、融资支持、信贷奖励有关。例如：“贷款贴息”“融资补助”“银行贷款利息补贴”。
10. 其他：不属于以上任何一类的项目。

请输出严格的 JSON 格式结果（不要包含额外文字或说明）：

{
  "project": "",
  "judgement_result": "",  // 从上面10类中选择一个类型
  "reason": ""         // 简要说明分类依据（不超过30字）
}

项目名称：
'''

In [ ]:
def my_rule(project_name):
    """
    调用大模型，根据项目名称返回判断结果与理由。
    假设模型返回 JSON 格式字符串，例如：
    {"判断结果": "税收相关", "reason": "名称中含有‘税’字"}
    """
    # try:
        # 构造输入（这里你之后可以自定义 prompt）
    if project_name == "合计":
        return {
            "judgement_result": "其它",
            "reason": "合计"
        }

        
    prompt = PROMPT_CLASSIFY + project_name

    # 调用大模型（用最便宜的）
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "你是一个财务分类助手。"},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
    )

    # 提取返回文本
    content = response.choices[0].message.content.strip()


    # 解析JSON
    result = json.loads(content)

    # 返回一个 dict 以便多列合并
    return {
        "判断结果": result.get("judgement_result", "未知"),
        "reason": result.get("reason", "未提供理由")
    }

    # except Exception as e:
    #     # 错误处理（避免 apply 出错中断）
    #     return {
    #         "判断结果": "调用出错",
    #         "reason": str(e)
    #     }



In [32]:
start_time = time.time()  
df_short = df_short.join(df_short['项目'].apply(lambda x: pd.Series(my_rule(x))))
end_time = time.time() 
print(f"耗时：{end_time - start_time:.2f} 秒")


TypeError: 'ChatCompletion' object is not subscriptable

In [ ]:
df_short.head()

,证券代码,证券简称,统计截止日期,列报会计科目,报表类型,项目,本期计入当期损益的金额,上期计入当期损益的金额,与资产相关/与收益相关,判断结果,reason
1,6,深振业A,2015-12-31,2,1,长沙市岳麓区财政局税收奖励金,1660000.0,NaN,NaN,其他,未匹配关键字
2,6,深振业A,2015-12-31,2,1,西安浐灞生态区生态宜居示范单位奖励资金,150000.0,NaN,NaN,其他,未匹配关键字
3,6,深振业A,2015-12-31,2,1,合计,2076250.0,210000.0,NaN,其他,未匹配关键字
4,6,深振业A,2016-12-31,2,1,长沙市岳麓区天顶街道办事处税收奖励金,NaN,266250.0,NaN,其他,未匹配关键字
5,6,深振业A,2016-12-31,2,1,长沙市岳麓区财政局税收奖励金,NaN,1660000.0,NaN,其他,未匹配关键字
